# Understanding HybridAGI prompting

In this notebook we are going to show you how HybridAGI build the prompts for the LLM, that way you can understand how exactly the Agent use tools. To do so, we are going to make the simplest agent possible and inspect the LLM calls.

In [1]:
import dspy
import hybridagi.core.graph_program as gp
from hybridagi.memory.integration.local import LocalProgramMemory
from hybridagi.core.datatypes import AgentState, Query
from hybridagi.modules.agents import GraphInterpreterAgent
from hybridagi.modules.agents.tools import SpeakTool

main = gp.GraphProgram(
	name = "main",
	description = "The main program",
)

main.add(gp.Action(
    id = "answer",
	tool = "Speak",
	purpose = "Answer the user question",
	prompt = "Please answer to the Objective's question",
))

main.connect("start", "answer")
main.connect("answer", "end")

# We build the program to verify its structure
main.build()

program_memory = LocalProgramMemory(index_name="prompting")

program_memory.update(main)

agent_state = AgentState()

tools = [
    SpeakTool(
        agent_state = agent_state
    ),
]

rag_agent = GraphInterpreterAgent(
    program_memory = program_memory,
    agent_state = agent_state,
    tools = tools,
    verbose = False,
)

# We can now setup the LLM using Ollama client from DSPy

lm = dspy.OllamaLocal(model='mistral', max_tokens=1024, stop=["\n\n\n"])
dspy.configure(lm=lm)

# And call our agent

result = rag_agent(Query(query="What is the city of the spatial and robotics industry in France?"))

print(lm.history[-1]["prompt"])



/home/yoan/.cache/pypoetry/virtualenvs/hybridagi-B1GoJrSC-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Given the fields `objective`, `context`, `purpose`, `prompt`, produce the fields `message`.

---

Follow the following format.

Objective: The long-term objective (what you are doing)

Context: The previous actions (what you have done)

Purpose: The purpose of the action (what you have to do now)

Prompt: The action specific instructions (How to do it)

Message: The message to send to the user

---

Objective: What is the city of the spatial and robotics industry in France?

Context:
 --- Step 0 ---
Call Program: main
Program Purpose: What is the city of the spatial and robotics industry in France?

Purpose: Answer the user question

Prompt: Please answer to the Objective's question

Message:
